In [21]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/planar_quadruped_landing/Project.toml`


In [22]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

reference_trajectory

## Reference Trajectory

## Problem Definition

In [23]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -4.0                   # yb_dot
xinit[12] = -4.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [24]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

In [32]:
nlp.cinds

3-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448

In [29]:
# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   248192
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      448
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-0.24999999992484773, 0.4035533906023351, 0.1745329251990924, -3.877603542002777e-11, -1.4222136526216301e-11, -0.5000000000015933, 0.050000000002085404, 7.710524253005343e-12, -3.999999999994787, -2.012770699685609e-13  …  -7.069100603476383e-14, -0.5000000000583055, -7.422611969180345e-14, -5.051745458798031e-12, 2.1436398829733497e-12, -4.780545591816849e-15, 2.9897229344460088e-12, -1.8363438799711354e-13, 1.0383583069873222e-11, -1.8313271056116362e-13], Ipopt.Optimizer)

In [30]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  7.515227329335517e-11
  9.06130726008314e-12
 -3.4056091280376677e-13
 -3.877603542002777e-11
 -1.4222136526216301e-11
 -1.5932810626395622e-12
  2.0854012960924706e-12
  7.710524253005343e-12
  5.213163234429885e-12
 -2.012770699685609e-13
 -4.874839547071352e-12
 -1.0462741784067475e-12
 -2.5514488055611164e-13
 -2.794061409933385e-14

In [31]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
  4.1594228061825333e-11
 -1.75621739373355e-11
  3.867608386545735e-13
 -1.8101857985140935e-11
 -7.069100603476383e-14
 -5.830547156193688e-11
 -7.422611969180345e-14
 -5.051745458798031e-12
  2.1436398829733497e-12
 -4.780545591816849e-15
  2.9897229344460088e-12
 -1.8363438799711354e-13
  1.0383583069873222e-11
 -1.8313271056116362e-13